In [23]:
import os
import json
import matplotlib.pyplot as plt
from nltk import ngrams






# Combine all the problems into a single string
all_problems_text = ' '.join(problems_list)



In [3]:
!wordloud --version

'wordloud' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
all_problems_text

'cleanliness staff friendliness staff friendliness privacy privacy cleanliness privacy cleanliness checkout privacy amenities staff friendliness room service checkout cleanliness food quality amenities food quality noisy neighbors privacy food quality privacy cleanliness noisy neighbors pricing checkout privacy food quality clean towels cleanliness food quality noisy neighbors noisy neighbors cleanliness pricing bad smell pricing noisy neighbors food quality privacy room service bad smell pricing checkout amenities pricing amenities noisy neighbors pricing amenities cleanliness noisy neighbors noisy neighbors amenities bad smell staff friendliness clean towels room service room service cleanliness pricing pricing pricing clean towels cleanliness room service pricing food quality clean towels bad smell food quality bad smell noisy neighbors checkout noisy neighbors checkout staff friendliness bad smell amenities privacy staff friendliness noisy neighbors food quality noisy neighbors cle

In [24]:
# problems = all_problems_text.split()

# # Count the frequency of each individual problem (unigrams)
# problem_counts = Counter(problems)

# # Get the 10 most common individual problems (unigrams)
# most_common_problems = problem_counts.most_common(10)

# # Separate the problems and their corresponding frequencies for unigrams
# unigram_labels, unigram_frequencies = zip(*most_common_problems)

# # Generate bigrams
# bigrams = list(ngrams(problems, 2))
# bigram_counts = Counter(bigrams)

# # Get the 10 most common bigrams
# most_common_bigrams = bigram_counts.most_common(10)

# # Separate the bigrams and their corresponding frequencies
# bigram_labels, bigram_frequencies = zip(*most_common_bigrams)

# # Format bigrams for plotting
# bigram_labels = [' '.join(bigram) for bigram in bigram_labels]

# # Combine the results for plotting
# combined_labels = list(unigram_labels) + bigram_labels
# combined_frequencies = list(unigram_frequencies) + list(bigram_frequencies)

# # Plot the combined results
# plt.figure(figsize=(12, 8))
# plt.barh(combined_labels, combined_frequencies, color=['skyblue'] * len(unigram_labels) + ['lightgreen'] * len(bigram_labels))
# plt.xlabel('Frequency')
# plt.title('Top 10 Most Common Unigrams and Bigrams')
# plt.gca().invert_yaxis()  # To display the highest frequency at the top
# plt.tight_layout()
# plt.show()

In [25]:
print(len(os.listdir("Vcon")))

88


In [26]:
len(problems_list)

88

In [27]:
import re
from collections import defaultdict
from difflib import get_close_matches


def gather_problems(folder_path):
    
    problems_list = []
    # Iterate through each file in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):  
            file_path = os.path.join(folder_path, filename)
            # Open and load the JSON file
            with open(file_path, 'r') as file:
                data = json.load(file)
                # Check if the "attachments" field exists and contains the problem
                if "attachments" in data and len(data["attachments"]) > 0:
                    problem = data["attachments"][0]["body"]["problem"]
                    problems_list.append(problem)




def add_variations(word_set):
    
        categories = {
        "Service Quality": [],
        "Cleanliness": [],
        "Food and Amenities": [],
        "Noise and Privacy": [],
        "Check-in and Check-out": [],
        "Price": [],
        "Other": []
    }

    # Define synonyms (expanded and refined)
    synonyms = {
        "service": ["assistance", "help", "support", "care", "attention", "hospitality"],
        "dirty": ["filthy", "unclean", "messy", "grimy", "soiled", "unhygienic", "smelly", "odor"],
        "noise": ["racket", "commotion", "disturbance", "clamor", "din", "loud", "noisy"],
        "expensive": ["costly", "pricey", "overpriced", "exorbitant", "extravagant", "high"],
        "staff": ["employee", "worker", "personnel", "attendant", "assistant", "receptionist"],
        "bed": ["mattress", "cot", "bunk", "berth", "sleeper", "pillow", "blanket"],
        "shower": ["bath", "washroom", "bathroom", "lavatory", "tub", "toilet", "sink"],
        "internet": ["wifi", "wi-fi", "connection", "network", "broadband", "online"],
        "clean": ["spotless", "immaculate", "pristine", "sanitary", "tidy", "hygienic"],
        "rude": ["impolite", "discourteous", "disrespectful", "uncivil", "ill-mannered", "unfriendly"],
        "helpful": ["supportive", "accommodating", "cooperative", "obliging", "considerate", "friendly"],
        "quiet": ["silent", "peaceful", "tranquil", "serene", "hushed", "calm"],
        "privacy": ["seclusion", "solitude", "isolation", "confidentiality", "discretion", "personal space"],
        "check": ["verify", "inspect", "examine", "review", "assess", "process"],
        "wait": ["delay", "pause", "hold", "linger", "tarry", "queue"],
        "price": ["cost", "charge", "fee", "rate", "fare", "pricing", "expensive", "cheap"],
        "food": ["meal", "breakfast", "lunch", "dinner", "cuisine", "dish", "menu"],
        "smell": ["odor", "scent", "aroma", "stench", "fragrance"],
        "quality": ["standard", "grade", "caliber", "condition", "value"],
    }

    
    variations = set()
    for word in word_set:
        variations.add(word)
        variations.add(word + "s")  
        variations.add(word + "ed") 
        variations.add(word + "ing")  
    return variations




def categorize_complaints(problem_list):
    
    category_keywords = {
    "Service Quality": {"service", "staff", "rude", "helpful", "attentive", "friendly", "unfriendly", "hospitality"},
    "Cleanliness": {"clean", "dirty", "dust", "stain", "hygiene", "smell", "odor"},
    "Food - Amenities": {"amenity", "facility", "wifi", "internet", "shower", "bed", "food", "breakfast", "meal"},
    "Noise and Privacy": {"noise", "quiet", "privacy", "disturb", "loud"},
    "Check-in and Check-out": {"check", "reception", "lobby", "wait", "key", "process"},
    "Price": {"price", "expensive", "cheap", "value", "cost", "charge", "fee", "rate"},
}

for category, keywords in category_keywords.items():
    expanded_keywords = set()
    for word in keywords:
        expanded_keywords.update(add_variations({word}))
        if word in synonyms:
            expanded_keywords.update(add_variations(set(synonyms[word])))
    category_keywords[category] = expanded_keywords
    categorized_complaints = defaultdict(list)

    for problem in problem_list:
        # Lowercase and remove extra spaces
        problem_cleaned = problem.lower().strip()
        
        matched = False
        for category, keywords in category_keywords.items():
            # Check if the cleaned problem contains any keyword directly
            if any(keyword in problem_cleaned for keyword in keywords):
                categorized_complaints[category].append(problem)
                matched = True
                break  # If matched, no need to check other categories
        
        # Add to "Other" if it doesn't match any category
        if not matched:
            categorized_complaints["Other"].append(problem)

    # Sort the complaints in each category (keeping repetitions)
    for category in categorized_complaints:
        categorized_complaints[category] = sorted(categorized_complaints[category])

    return dict(categorized_complaints)


def categorize_and_rename_in_directory(problems_list, directory):
    categorized_complaints = defaultdict(list)
    file_category_map = defaultdict(list)  # Track which files belong to which category

    # Get all files in the directory
    files = os.listdir(directory)
    
    if len(problems_list) != len(files):
        raise ValueError("The number of problems does not match the number of files in the directory.")

    # Process each problem and corresponding file
    for problem, file in zip(problems_list, files):
        problem_cleaned = problem.lower().strip()

        matched = False
        for category, keywords in category_keywords.items():
            # Check if the cleaned problem contains any keyword
            if any(keyword in problem_cleaned for keyword in keywords):
                categorized_complaints[category].append(problem)
                file_category_map[category].append(file)
                matched = True
                break
        
        # If no match, classify under "Other"
        if not matched:
            categorized_complaints["Other"].append(problem)
            file_category_map["Other"].append(file)

    # Keep track of how many files we've renamed for each category
    rename_count = defaultdict(int)

    # Dictionary to store the renamed files
    renamed_files = {}

    # Renaming logic based on categorized complaints
    for category, files in file_category_map.items():
        unique_files = set(files)  # Avoid renaming the same file multiple times
        for original_file in unique_files:
            rename_count[category] += 1
            new_filename = f"{category.lower().replace(' ', '_')}{rename_count[category]}.json"
            
            # Construct full paths
            original_file_path = os.path.join(directory, original_file)
            new_file_path = os.path.join(directory, new_filename)
            
            try:
                if os.path.exists(original_file_path):
                    os.rename(original_file_path, new_file_path)
                    renamed_files[original_file] = new_filename
                else:
                    renamed_files[original_file] = "File not found"  # Handle missing files
            except:
                continue

    return categorized_complaints, renamed_files






{'Check-in and Check-out': ['checkout', 'checkout', 'checkout', 'checkout', 'checkout', 'checkout', 'checkout'], 'Other': ['amenities', 'amenities', 'amenities', 'amenities', 'amenities', 'amenities', 'amenities'], 'Food - Amenities': ['food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality'], 'Price': ['pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing'], 'Cleanliness': ['bad smell', 'bad smell', 'bad smell', 'bad smell', 'bad smell', 'bad smell', 'clean towels', 'clean towels', 'clean towels', 'clean towels', 'clean towels', 'clean towels', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness', 'cleanliness'], 'Noise and Privacy': ['noisy neighbors', 'noisy neighbors', 'noisy neighbors', 'noisy neighbors', 'noisy neighbors', 'noisy neighbors', 'noisy nei

In [29]:
def categorize_and_rename_in_directory(problems_list, directory):
    categorized_complaints = defaultdict(list)
    file_category_map = defaultdict(list)  # Track which files belong to which category

    # Get all files in the directory
    files = os.listdir(directory)
    
    if len(problems_list) != len(files):
        raise ValueError("The number of problems does not match the number of files in the directory.")

    # Process each problem and corresponding file
    for problem, file in zip(problems_list, files):
        problem_cleaned = problem.lower().strip()

        matched = False
        for category, keywords in category_keywords.items():
            # Check if the cleaned problem contains any keyword
            if any(keyword in problem_cleaned for keyword in keywords):
                categorized_complaints[category].append(problem)
                file_category_map[category].append(file)
                matched = True
                break
        
        # If no match, classify under "Other"
        if not matched:
            categorized_complaints["Other"].append(problem)
            file_category_map["Other"].append(file)

    # Keep track of how many files we've renamed for each category
    rename_count = defaultdict(int)

    # Dictionary to store the renamed files
    renamed_files = {}

    # Renaming logic based on categorized complaints
    for category, files in file_category_map.items():
        unique_files = set(files)  # Avoid renaming the same file multiple times
        for original_file in unique_files:
            rename_count[category] += 1
            new_filename = f"{category.lower().replace(' ', '_')}{rename_count[category]}.json"
            
            # Construct full paths
            original_file_path = os.path.join(directory, original_file)
            new_file_path = os.path.join(directory, new_filename)
            
            try:
                if os.path.exists(original_file_path):
                    os.rename(original_file_path, new_file_path)
                    renamed_files[original_file] = new_filename
                else:
                    renamed_files[original_file] = "File not found"  # Handle missing files
            except:
                continue

    return categorized_complaints, renamed_files


directory_path = "Vcon"

# Categorize and rename
categorized_result, renamed_files = categorize_and_rename_in_directory(problems_list, directory_path)


Categorized Complaints:
defaultdict(<class 'list'>, {'Check-in and Check-out': ['checkout', 'checkout', 'checkout', 'checkout', 'checkout', 'checkout', 'checkout'], 'Other': ['amenities', 'amenities', 'amenities', 'amenities', 'amenities', 'amenities', 'amenities'], 'Food - Amenities': ['food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality', 'food quality'], 'Price': ['pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing', 'pricing'], 'Cleanliness': ['cleanliness', 'cleanliness', 'clean towels', 'cleanliness', 'bad smell', 'clean towels', 'cleanliness', 'cleanliness', 'clean towels', 'bad smell', 'bad smell', 'cleanliness', 'cleanliness', 'clean towels', 'bad smell', 'bad smell', 'bad smell', 'cleanliness', 'cleanliness', 'clean towels', 'cleanliness', 'clean towels'], 'Noise and Privacy': ['noisy neighbors', 'noisy neighbors', 'privacy', 'noisy neighbors', 'noi

In [32]:
filenames=os.listdir("vcon")

In [33]:
filenames

['check-in_and_check-out1.json',
 'check-in_and_check-out2.json',
 'check-in_and_check-out3.json',
 'check-in_and_check-out4.json',
 'check-in_and_check-out5.json',
 'check-in_and_check-out6.json',
 'check-in_and_check-out7.json',
 'cleanliness1.json',
 'cleanliness10.json',
 'cleanliness11.json',
 'cleanliness12.json',
 'cleanliness13.json',
 'cleanliness14.json',
 'cleanliness15.json',
 'cleanliness16.json',
 'cleanliness17.json',
 'cleanliness18.json',
 'cleanliness19.json',
 'cleanliness2.json',
 'cleanliness20.json',
 'cleanliness21.json',
 'cleanliness22.json',
 'cleanliness3.json',
 'cleanliness4.json',
 'cleanliness5.json',
 'cleanliness6.json',
 'cleanliness7.json',
 'cleanliness8.json',
 'cleanliness9.json',
 'food_-_amenities1.json',
 'food_-_amenities2.json',
 'food_-_amenities3.json',
 'food_-_amenities4.json',
 'food_-_amenities5.json',
 'food_-_amenities6.json',
 'food_-_amenities7.json',
 'food_-_amenities8.json',
 'food_-_amenities9.json',
 'noise_and_privacy1.json',
 